In [ ]:
import datetime
import pandas as pd
import micromegas
pd.set_option('display.max_rows', 500)
client = micromegas.connect()

In [ ]:
%%time
sql = """
SELECT process_id, "processes.exe" as exe, SUM(payload_size) as sum_payload_size
FROM blocks
WHERE array_has(blocks."streams.tags", 'cpu')
GROUP by process_id, exe
ORDER BY 2 DESC
LIMIT 10
"""
client.query(sql)

In [ ]:
%%time
sql = """
SELECT *
FROM processes
WHERE exe ilike '%flight%'
order by start_time DESC
limit 2
"""
client.query(sql)

In [ ]:
sql = """
      WITH begin_events AS (
            SELECT span_id, time as begin_time, name, filename, target, line
            FROM view_instance('async_events', '{process_id}')
            WHERE event_type = 'begin'
        ),
        end_events AS (
            SELECT span_id, time as end_time
            FROM view_instance('async_events', '{process_id}')
            WHERE event_type = 'end'
        )
        SELECT 
            b.span_id,
            b.begin_time,
            e.end_time,
            b.name,
            b.filename,
            b.target,
            b.line
        FROM begin_events b
        INNER JOIN end_events e ON b.span_id = e.span_id
        ORDER BY b.begin_time
  ;""".format( process_id = "d35a4b40-7407-40cd-a1cf-c619124ee529" )
df = client.query(sql)

In [ ]:
df["duration"] = df["end_time"] - df["begin_time"]

In [ ]:
df["duration"].min()